In [2]:
%cd ..

d:\project\deploy-chest-X-ray


In [15]:
pip install  transformers


     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 131.3 kB/s eta 0:00:01
     ------------------ ------------------- 20.5/42.0 kB 131.3 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.0 kB 119.8 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 135.6 kB/s eta 0:00:00
  Using cached regex-2025.7.34-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB 330.3 kB/s eta 0:00:35
   ---------------------------------------- 0.0/11.3 MB 281.8 kB/s eta 0:00:40
   -------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from PIL import Image
import pandas as pd

df = pd.read_csv("./data_map.csv")
df.head()

,image,class,split,resized
0,data\test\NORMAL\IM-0001-0001.jpeg,NORMAL,test,data/resize/test_NORMAL_IM-0001-0001.jpeg
1,data\test\NORMAL\IM-0003-0001.jpeg,NORMAL,test,data/resize/test_NORMAL_IM-0003-0001.jpeg
2,data\test\NORMAL\IM-0005-0001.jpeg,NORMAL,test,data/resize/test_NORMAL_IM-0005-0001.jpeg
3,data\test\NORMAL\IM-0006-0001.jpeg,NORMAL,test,data/resize/test_NORMAL_IM-0006-0001.jpeg
4,data\test\NORMAL\IM-0007-0001.jpeg,NORMAL,test,data/resize/test_NORMAL_IM-0007-0001.jpeg


In [4]:
from sklearn.preprocessing import LabelEncoder
import pickle

lbl_encoder = LabelEncoder()

df['clas_enc'] = lbl_encoder.fit_transform(df['class'])

with open("data/lbl_encoder.pkl","wb") as f:
    pickle.dump(lbl_encoder,f)

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

In [6]:
class XrayData(Dataset):
    def __init__(self, transform, df_split):

        self.transform = transform
        self.df_split = df_split

    def __len__(self):
        return len(self.df_split)
    
    def __getitem__(self, index):
        
        img_path = self.df_split.iloc[index]["resized"]
        clas = self.df_split.iloc[index]["clas_enc"]
        img = Image.open(img_path).convert("L")
        img_trans = self.transform(img)

        return img_trans, clas 
        

In [7]:
train_data = XrayData(df_split=df[df['split']=="train"], transform=transforms.ToTensor())
val_data = XrayData(df_split=df[df['split']=="val"], transform=transforms.ToTensor())

In [8]:
train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
val_loader = DataLoader(val_data,batch_size=32,shuffle=True)

In [10]:
model = models.resnet18(weights = models.ResNet18_Weights.IMAGENET1K_V1)

In [10]:
model.conv1 = nn.Conv2d(1,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)
model.fc = nn.Linear(in_features=512,out_features=2,bias=True)

In [11]:
optimizer = torch.optim.Adam(model.parameters())
loss = nn.CrossEntropyLoss()

In [14]:
from tqdm import tqdm
for epoqe in range(7):

    model.train()
    running_loss = 0

    for img, lbls in tqdm(train_loader,total=len(train_loader)):

        # img, lbls = 
        optimizer.zero_grad()
        preds = model(img)
        loss_val = loss(preds,lbls)
        loss_val.backward()
        optimizer.step()
        running_loss += loss_val.item()

    print(running_loss / len(train_loader))

100%|██████████| 84/84 [04:48<00:00,  3.43s/it]


0.025031662201702904


100%|██████████| 84/84 [04:11<00:00,  2.99s/it]


0.012773815878019724


100%|██████████| 84/84 [04:31<00:00,  3.23s/it]


...

0.008148803672936213


100%|██████████| 84/84 [05:27<00:00,  3.90s/it]


0.00541712880833854


100%|██████████| 84/84 [05:31<00:00,  3.95s/it]

0.009740231092629463


In [ ]:
torch.save(model.state_dict(),"./model.pth")

In [15]:
from torchvision.models import efficientnet_b0

model2 = efficientnet_b0(pretrained=True)

model2.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
model2.classifier[1] = nn.Linear(model2.classifier[1].in_features, 2)
optimizer = torch.optim.Adam(model2.parameters())

In [16]:
model2

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [17]:
from tqdm import tqdm
for epoqe in range(7):

    model2.train()
    running_loss = 0

    for img, lbls in tqdm(train_loader,total=len(train_loader)):

        # img, lbls = 
        optimizer.zero_grad()
        preds = model2(img)
        loss_val = loss(preds,lbls)
        loss_val.backward()
        optimizer.step()
        running_loss += loss_val.item()

    print(running_loss / len(train_loader))

100%|██████████| 84/84 [05:11<00:00,  3.71s/it]


0.15996802411973476


100%|██████████| 84/84 [04:14<00:00,  3.03s/it]


0.09658319167127567


100%|██████████| 84/84 [04:15<00:00,  3.04s/it]


...

0.029829528882796046


100%|██████████| 84/84 [05:22<00:00,  3.84s/it]


0.03782774511983873


100%|██████████| 84/84 [07:15<00:00,  5.19s/it]

0.031099243767953123


In [18]:
torch.save(model2.state_dict(),"./model2.pth")

In [21]:
from torchvision.models import mobilenet_v2

model3 = mobilenet_v2(pretrained=True)

model3.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
model3.classifier[1] = nn.Linear(model3.classifier[1].in_features, 2)
optimizer = torch.optim.Adam(model3.parameters())

c:\Users\Moaaz Reda\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Moaaz Reda\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
model3

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [23]:
from tqdm import tqdm
for epoqe in range(5):

    model3.train()
    running_loss = 0

    for img, lbls in tqdm(train_loader,total=len(train_loader)):

        # img, lbls = 
        optimizer.zero_grad()
        preds = model3(img)
        loss_val = loss(preds,lbls)
        loss_val.backward()
        optimizer.step()
        running_loss += loss_val.item()

    print(running_loss / len(train_loader))

100%|██████████| 84/84 [03:53<00:00,  2.78s/it]


0.1772746684007524


100%|██████████| 84/84 [04:36<00:00,  3.30s/it]


0.07764775018828611


100%|██████████| 84/84 [04:05<00:00,  2.93s/it]


0.058538675194584014


100%|██████████| 84/84 [03:59<00:00,  2.85s/it]


0.07733196620773967


100%|██████████| 84/84 [04:00<00:00,  2.86s/it]

0.024013825485737817


In [24]:
torch.save(model3.state_dict(),"./model3.pth")

In [ ]:
def infer(img):
    img_gray = img.convert("L")
    img_trans = transfrorms.To_Tensor()(img_gray).unsqueeze(0).to(device)
    preds = model(img_trans)
    clas_index = torch.argmax(preds).item()
    final_pred = lbl_encoder.inverse_transform([clas_index])[0]
    return final_pred
    